In [1]:
import prepare_ml_dataset


/home/jliao/anaconda3/envs/notebook/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-03-25 17:16:23.602085: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-03-25 17:16:23.610744: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-25 17:16:23.688679: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-03-25 17:16:23.758703: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to r

In [ ]:
# # here we run through calculate for all ml dataset as set in 
# import importlib
# importlib.reload(prepare_ml_dataset)

# prepare_ml_dataset.prepare_ml_dataset_batch()

In [ ]:
import importlib
importlib.reload(prepare_ml_dataset)

raw_feature_names =  ['symh','asyh','ae','asyd'] #['symh','asyh','asyd','ae','f10.7','kp','swp','swn','swv','by','bz']

# [51767.680, 44428.696, 38130.120, 32724.498, 28085.268, 24103.668, 20686.558, 17753.876, 15236.896, 13076.798, 11222.936, 9631.899, 8266.406, 7094.516, 6088.722, 5225.528, 4484.742
# , 3848.919, 3303.284, 2834.964, 2433.055, 2088.129, 1792.096, 1538.062, 1319.977, 1132.846, 972.237]
number_history_arr = [7,8]
forecast_arr = ["all", "index","none"]
dL01_arr = [True, False]
species_arr = ['h', 'o']
energy_arr = ['972237', '51767680']  

for number_history in number_history_arr:
    for forecast in forecast_arr:
       for dL01 in dL01_arr:
           for species in species_arr:
               for energy in energy_arr:
               
                    prepare_ml_dataset.load_ml_dataset(energy, species, recalc = True, plot_data = False, save_data = False
                                                          , dL01=dL01, forecast = forecast, number_history =number_history, raw_feature_names =  raw_feature_names)


In [ ]:
# import prepare_ml_dataset
# import importlib
# importlib.reload(prepare_ml_dataset)

# raw_feature_names =  ['symh','asyh','ae','asyd'] #['symh','asyh','asyd','ae','f10.7','kp','swp','swn','swv','by','bz']

# # [51767.680, 44428.696, 38130.120, 32724.498, 28085.268, 24103.668, 20686.558, 17753.876, 15236.896, 13076.798, 11222.936, 9631.899, 8266.406, 7094.516, 6088.722, 5225.528, 4484.742
# # , 3848.919, 3303.284, 2834.964, 2433.055, 2088.129, 1792.096, 1538.062, 1319.977, 1132.846, 972.237]
# number_history_arr = [7]
# forecast_arr = ["all"]
# dL01_arr = [True]
# species_arr = ['h']
# energy_arr = ['972237']  

# for number_history in number_history_arr:
#     for forecast in forecast_arr:
#        for dL01 in dL01_arr:
#            for species in species_arr:
#                for energy in energy_arr:
               
#                     prepare_ml_dataset.prepare_ml_dataset(energy, species, recalc = True, plot_data = True, save_data = False
#                                                           , dL01=dL01, forecast = forecast, number_history =number_history, raw_feature_names =  raw_feature_names)


In [ ]:
import prepare_fulldata
import importlib
import initialize_var
import prepare_ml_dataset

importlib.reload(prepare_fulldata)
importlib.reload(prepare_ml_dataset)
importlib.reload(initialize_var)

recalc = False
plot_data = True
save_data = True

raw_feature_names =  ['symh']#,'asyh','ae','asyd'] #['symh','asyh','asyd','ae','f10.7','kp','swp','swn','swv','by','bz']
number_history = 7
energy = '972237'
species = 'h'
forecast = "none"
dL01 = True
test_ts = '2017-01-01'
test_te = '2018-01-01'

dataset_csv, data_settings, directories = initialize_var.initialize_data_var(energy=energy, species=species, raw_feature_names = raw_feature_names, forecast = forecast, number_history = number_history, test_ts=test_ts, test_te=test_te, dL01=dL01)
        
df_data, directories, fulldataset_csv, fulldata_settings = prepare_fulldata.load_fulldata(energy, species, recalc = recalc, raw_feature_names = raw_feature_names, number_history = number_history, save_data = save_data, plot_data = plot_data)
        
df_full = prepare_fulldata.read_probes_data(directories["rawdata_dir"], fulldata_settings)

df_data[[fulldata_settings['doubletime_name']]] = df_full[[fulldata_settings['doubletime_name']]]

# df_data[[fulldata_settings['doubletime_name']]+fulldata_settings["raw_coor_names"]+ fulldata_settings["raw_feature_names"]] = df_full[[fulldata_settings['doubletime_name']]+fulldata_settings["raw_coor_names"]+ fulldata_settings["raw_feature_names"]]

index_good = get_good_index(df_full, data_settings, fulldata_settings)

if data_settings["forecast"] == "all":
    data_settings["feature_history_names"] = remove_features_by_time(fulldata_settings["feature_history_names"], "*_0h")
elif data_settings["forecast"] == "index":
    data_settings["feature_history_names"] = remove_index_features_by_time(fulldata_settings["feature_history_names"], "*_0h")
else:
    data_settings["feature_history_names"] = fulldata_settings["feature_history_names"]            

df_data = df_data.loc[index_good,[fulldata_settings['doubletime_name'], fulldata_settings['datetime_name'],data_settings['y_name'], data_settings['log_y_name']]+ fulldata_settings['coor_names']+fulldata_settings['feature_history_names']]

df_full = df_full.loc[index_good, :]

#set test set. Here we use one year (2017) of data for test set 
index_train, index_valid, index_test = create_ml_indexes(df_data,  fulldata_settings, data_settings["test_ts"], data_settings["test_te"])

# Each round, one can only train one y. If train more than one y, need to  repeat from here
x_train, x_valid, x_test, y_train, y_valid, y_test = create_ml_data(df_data, index_train, index_valid, index_test, data_settings["log_y_name"], fulldata_settings["coor_names"], data_settings["feature_history_names"])  

print("shapes of x_train, x_valid, x_test, y_train, y_valid, y_test ")
print(x_train.shape, x_valid.shape, x_test.shape, y_train.shape, y_valid.shape, y_test.shape)

if save_data:
    save_df_data(df_full.loc[index_good, [fulldata_settings['datetime_name'], data_settings["y_name"]] + fulldata_settings["raw_coor_names"] + fulldata_settings["raw_feature_names"]], index_train, index_valid, index_test, dataset_csv)

    save_csv_data(x_train, x_valid, x_test, y_train, y_valid, y_test , dataset_csv)
    
if plot_data:
    plot_y_data(df_data.loc[index_good, [ fulldata_settings['datetime_name'],  data_settings["y_name"],data_settings["log_y_name"]]], data_settings["y_name"],data_settings["log_y_name"],  fulldata_settings['datetime_name'], fulldataset_csv["df_y"]+ data_settings["log_y_name"])
    
    plot_coor_data(df_data.loc[index_good, [fulldata_settings['datetime_name'],fulldata_settings["coor_names"]  ]], fulldata_settings["coor_names"],  fulldata_settings['datetime_name'], fulldataset_csv["df_coor"])
                
    plot_feature_data(df_data.loc[index_good, [fulldata_settings['datetime_name'],fulldata_settings["feature_names"] ]], fulldata_settings["feature_names"],  fulldata_settings['datetime_name'], fulldataset_csv["df_coor"])
